In [66]:
import pandas as ps
import numpy as np
import random
import math

In [67]:
class Config():
    T = 1000  # time
    N = 100   # number of banks
    S = 10
    α = 0.3   # alpha, liquidation cost of collaterals
    φ = 0.1   # phi    attractiveness
    δ = 0.015 # delta  screening cost
    σ = 1.5   # sigma 
    depositRate = 0.03
    riskfree = 0.01
    RANDOM_CONNECTIVITY = 0.75
    TOT_SIMULATIONS = 4
    τ = 1     # tau
    η = 0.09  #  eta 
    prud = 0.045
    u = 1.0   #  ypsilon 
    
    
    iT_inv = 500
    T_inv = float(iT_inv)

    # banks parameters
    initial_loan = 120.0
    initial_liquidity = 30.0
    initial_deposit = 130.0
    initial_equity = initial_liquidity / 2
    initial_probfail = 0.2
    initial_price = 0.3
    initial_haircut = 0.3
    minInterestInterbank = 0.02
    
class Status():
    maxEquity = 0.0
    banks = []
    matched = []
    credit = []
    interestInterbank = []
    marketBank = []
    
    mark = 0
    
    def init(self):
        self.maxEquity = 0.0
        self.banks = []
        self.matched = np.zeros( (Config.N,Config.N) )
        self.interestInterbank = np.zeros( (Config.N,Config.N) )  
        self.credit = np.zeros( (Config.N,Config.N) )  
        self.marketBank = np.zeros( (Config.N,Config.N) )  
        for i in range(Config.N):
            self.banks.append( Bank() )
        

In [68]:
class Bank():
    def set_capacity():
        self.capacity = 1.0 - self.haircut*self.asset
        if self.capacity<1.0:
            self.capacity = 4.0;
        # fprintf( out49 ) **
        
    def determineProbFail():
        self.probFail = 1.0 - self.equity / Status.maxEquity
            
    def __init__(self):
        self.loan = Config.initial_loan
        self.liquidity = Config.initial_liquidity
        self.deposit = Config.initial_deposit
        self.equity = Config.initial_equity
        self.probfail = Config.initial_probfail
        self.haircut = Config.initial_haircut
        self.new_deposit = self.Dloan  = 0.0
        self.interbankLoan = 0.0
        self.interbankDebt = 0.0
        self.failB = 0.0
        self.asset = 0.0
        self.leverage = 0.0
        self.deltaD = self.firesale = self.crunch =  0.0
        
        self.asset = self.loan+ self.liquidity
        self.leverage = self.loan / self.equity
        
        self.rate = 0.0       # tasso
        self.totalRate = 0.0  # sommatasso
        self.connected = 0.0  # collegati
        
        self.marked = 0
        self.neighbour = 0   
        
        self.a = 0.0
        self.b = 0.0
        self.c = 0.0
        
        self.set_capacity()


In [71]:
def matrixRandom():
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        for j in range(len(Status.banks)):
            Status.marketBank[i][j] = 0.0

    for i in range(len(Status.banks)):
        if Status.banks[i].connected != 1:
            aux = random.random()
            if aux < Config.RANDOM_CONNECTIVITY:
                j=i
                while i==j:
                    j = int(random.random() * Config.N)
                    if i!=j:
                        Status.marketBank[i][j]=1
                        Status.banks[i].neighbour = j
                        Status.banks[i].connected = 1

    for i in range(len(Status.banks)):
        Status.banks[i].marked = -1
    for i in range(len(Status.banks)):
        if Status.banks[i].marked == -1:
            Status.mark += 1
            matrixRandom_dfs(i)
    
                        
def matrixRandom_dfs(n):
    if (Status.banks[i].marked == -1):
        Status.banks[i].marked = Status.mark
        for i in range(len(Status.banks)):
            if (Status.marketBank[n][i] == 1 or Status.marketBank[i][n] == 1) and Status.banks[i].marked == -1 :
                matrixRandom_dfs(i)

def matrixPreferential():
    k = 0
    switching = 0
    ref = 0.0
    fact = 0.0
    prob = 0.0
    fitness = 0.0
    liquidityMax = -1000.0
    interestMin = 1000.0
    i_max = -1
    i_max2= -1
    for j in range(len(Status.banks)):
        if Status.banks[j].liquidity > liquidityMax:
            liquidityMax = Status.banks[j].liquidity
            i_max = j
        if Status.banks[j].rate < interestMin:
            interestMin = Status.banks[j].rate
            i_max2= j

    β = np.zeros( (Config.N) )
    for i in range(len(Status.banks)):
        Status.banks[i].fit = 0.0
        Status.banks[i].outgoingLink = 0
        Status.banks[i].incomigLink = 0
        Status.banks[i].link = 0
    for j in range(len(Status.banks)):
        Status.banks[j].fit = \
            Config.u * Status.banks[j].liquidity / liquidityMax + (1-Config.u)* interestMin * Status.banks[j].rate 
        # fprintf( out20 ) **

    Status.marketBank = np.zeros( (Config.N,Config.N), int )
    for i in range(len(Status.banks)):
        β[i] = Config.T_inv
    for i in range(len(Status.banks)):
        j=i
        while j==i:
            j=int(random.random()*Config.N)
        k = System.banks[i].neighbour
        prob = random.random()
        fitness = System.banks[j].fit - System.banks[k].fit 
        ref = β[i] * fitness
        fact = 1.0 / (1.0 + math.exp(-ref))
        if (prob<fact) and (i!=j):
            switching += 1
            Status.marketBank[i][k] = 0
            Status.marketBank[i][j] = 1
            Status.banks[i].neighbour = j
        else:
            Status.marketBank[i][j] = 0
            Status.marketBank[i][k] = 1
            Status.banks[i].neighbour = k
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 1
    
    
    
def findMaxEquity():
    Status.maxEquity = 0.0
    for i in range(len(Status.banks)):
        if (Status.banks[i].equity>Status.maxEquity):
            Status.maxEquity = Status.banks[i].equity
            
def reviewBanksInterbankRate():
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not Status.credit[j][i]:
                Status.bank[j].a = Config.prud * Status.bank[i].asset   +  Config.δ *  Status.bank[i].asset 
                
                Status.bank[j].b = Status.bank[i].probfail * ( Config.α * Status.bank[i].asset -  Status.bank[i].capacity )
                
                Status.bank[j].c = ( 1 - Status.bank[i].probfail ) * Status.bank[i].capacity 
                
                Status.interestInterbank[j][i] = ( Status.bank[j].a - Status.bank[j].b ) / Status.bank[j].c
                if (Status.interestInterbank[j][i]< Config.minInterestInterbank):
                    Status.interestInterbank[j][i] = Config.minInterestInterbank

def reviewBanksRate():
    for i in range(len(Status.banks)):
        Status.banks[i].connected = 0.0
        Status.banks[i].totalRate = 0.0
        
    for j in range(len(Status.banks)):
        for i in range(len(Status.banks)):
            if not (Status.credit[j][i]):                
                Status.banks[j].connected += 1 
                Status.banks[j].totalRate += self.interestInterbank[j][i]
                Status.banks[j].rate = Status.banks[j].totalRate / Status.banks[j].connected
                

SyntaxError: invalid syntax (3731300589.py, line 60)

In [48]:
for simulation in range(Config.TOT_SIMULATIONS):
    Status.init()
    for t i range(Config.T):
        if (t<10):
            matrixRandom()
        else:
            
            for i in range(len(Status.banks)):
                Status.banks[i].setCapacity()
                findMaxEquity()
                Status.banks[i].determineProbFail()
                
            reviewBanksInterbankRate()
            reviewBanksRate()
            
            matrixPreferential()
                                

In [61]:
Status

__main__.Status

In [ ]:
Config.prud=0